In [7]:
import pandas as pd
!pip3 install pandas
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

df = pd.read_csv('data.csv')


ImportError: No module named 'pandas'

In [5]:
random.seed(42)

NameError: name 'random' is not defined

In [6]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df.shape

In [ ]:
df[df['y'] == 0].shape, df[df['y'] == 1].shape

In [ ]:
df1 = df[df['y'] == 1].sample(n = df[df['y'] == 0].shape[0])
df2 = df[df['y'] == 0]
df1.shape, df2.shape

In [ ]:
df = pd.concat([df1, df2])

In [ ]:
df = df.fillna(df.mean())

In [ ]:
df[df['y'] == 0].shape, df[df['y'] == 1].shape

In [ ]:
df = df.sample(frac = 1.)

In [ ]:
x = np.array(df.iloc[:, :-1])
y = np.array(df.iloc[:, -1:])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=21)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
sum(np.isnan(y_train))

In [ ]:
import sklearn.preprocessing as sp

scaler = sp.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

X_train[:10]

In [ ]:
import sklearn.linear_model as lm
import sklearn.metrics as sm
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import accuracy_score

N = [x for x in range(1000, 46000, 1000)]

ERRs = []

for n in N:
    LR = lm.LinearRegression()
    LR.fit(X_train[:n], y_train[:n])
    y_pred = LR.predict(X_test[:n])
    pos_feat = np.array([i for i in y_pred])
    ERRs.append(sm.mean_squared_error(pos_feat, y_test[:n]))

print('ROC-AUC: ', roc_auc_score(y_test, pos_feat))
pos_feat = [1 if i > 0.5 else 0 for i in pos_feat]
print('Accuracy: ', accuracy_score(y_test, pos_feat))
    
    
plt.plot(N, ERRs)
plt.xlabel("Nubmer of elements in a train set")
plt.ylabel("MSE")
plt.show()

In [ ]:
# plt.scatter(X_test[:, 1].shape, y_test.shape, c="black", alpha = 0.2)
# X_test[:, :1].shape, y_test.shape

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

N = [x for x in range(1000, 16000, 1000)]

ERRs = []

for n in N:
    KNR = KNeighborsRegressor()
    KNR.fit(X_train[:n], y_train[:n])
    y_pred = KNR.predict(X_test[:n])
    pos_feat = np.array([i for i in y_pred])
    ERRs.append(sm.mean_squared_error(pos_feat, y_test[:n]))

print('ROC-AUC: ', roc_auc_score(y_test[:n], pos_feat))
pos_feat = [1 if i > 0.5 else 0 for i in pos_feat]
print('Accuracy: ', accuracy_score(y_test[:n], pos_feat))
    
    
plt.plot(N, ERRs)
plt.xlabel("Nubmer of elements in a train set")
plt.ylabel("MSE")
plt.show()

In [ ]:
import sklearn.preprocessing as pp
PP = pp.PolynomialFeatures(degree=2)
X_new = PP.fit_transform(X_train)

X_test_new = PP.fit_transform(X_test)

N = [x for x in range(1000, 46000, 1000)]

ERRs = []

for n in N:
    LR = lm.LinearRegression()
    LR.fit(X_new[:n], y_train[:n])
    y_pred = LR.predict(X_test_new[:n])
    pos_feat = np.array([i for i in y_pred])
    ERRs.append(sm.mean_squared_error(pos_feat, y_test[:n]))

print('ROC-AUC: ', roc_auc_score(y_test, pos_feat))
pos_feat = [1 if i > 0.5 else 0 for i in pos_feat]
print('Accuracy: ', accuracy_score(y_test, pos_feat))
    
    
plt.plot(N, ERRs)
plt.xlabel("Nubmer of elements in a train set")
plt.ylabel("MSE")
plt.show()

In [ ]:
features = []
for elem in df:
     features.append(elem)

In [ ]:
features = features[1:]

In [ ]:
logit = lm.LogisticRegression()
logit.fit(X_train, y_train)
y_pred = logit.predict(X_test)
pos_feat = np.array([i for i in y_pred])

print('ROC-AUC: ', roc_auc_score(y_test, pos_feat))
print('Accuracy: ', accuracy_score(y_test, pos_feat))

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, threshold = roc_curve(y_test, pos_feat)
roc_auc = auc(fpr, tpr)

plt.title('ROC-curve')
plt.plot(fpr, tpr, 'r', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'g--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DTC = DecisionTreeClassifier(random_state=1)
DTC.fit(X_train, y_train)
y_pred = DTC.predict_proba(X_test)

pos_feat = np.array([i[1] for i in y_pred])

print('ROC-AUC: ', roc_auc_score(y_test, pos_feat))
print('Accuracy: ', accuracy_score(y_test, pos_feat))

In [ ]:
from sklearn.tree import export_graphviz
!pip install imageio
from imageio import imread

features = df.iloc[:, :-1]
feature_names = features.columns

def visualize_tree(tree, feature_names, fh=10, fw=10):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("/content/drive/My Drive/data/dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f, feature_names=feature_names)

    command = ["dot", "-Tpng", "/content/drive/My Drive/data/dt.dot", \
               "-o /content/drive/My Drive/data/dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")
    plt.figure(figsize=(fh, fw))
    plt.imshow(imread("/content/drive/My Drive/data/dt.png"))
    plt.axis("off")
    
visualize_tree(DTC, feature_names)

In [ ]:
np.random.seed(123)  # для одинакового бутстрапа в каждом запуске

summary = np.zeros((17034, 2))

DTC = DecisionTreeClassifier(random_state=1)

for i in range(100):
    choice = np.random.randint(0, 1400, size=len(X_train))
    X_train_boot = [X_train[n] for n in choice]
    y_train_boot = [y_train[n] for n in choice]
    DTC.fit(X_train_boot, y_train_boot)
    summary += DTC.predict_proba(X_test)

result = summary / 100

pos_feat = np.array([i[1] for i in result])

print('ROC-AUC: ', roc_auc_score(y_test, pos_feat))
print('Accuracy: ', accuracy_score(y_test, pos_feat))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)

forest.fit(X_train, y_train)

y_pred = forest.predict_proba(X_test)

pos_feat = np.array([i[1] for i in y_pred])

print('ROC-AUC: ', roc_auc_score(y_test, pos_feat))
pos_feat = [1 if i > 0.5 else 0 for i in pos_feat]
print('Accuracy: ', accuracy_score(y_test, pos_feat))